In [ ]:
# main.py

import os
import pandas as pd
import logging

from geomeppy import IDF

# Your geometry / fenestration / etc. modules
from geomz.building import create_building_with_roof_type
from fenez.fenestration import add_fenestration
from fenez.materials import update_construction_materials, assign_constructions_to_surfaces
from Elec.lighting import add_lights_and_parasitics
from DHW.water_heater import add_dhw_to_idf
from HVAC.custom_hvac import add_HVAC_Ideal_to_all_zones
from ventilation.add_ventilation import add_ventilation_to_idf
from setzone.add_outdoor_air_and_zone_sizing_to_all_zones import add_outdoor_air_and_zone_sizing_to_all_zones
from tempground.add_ground_temperatures import add_ground_temperatures
from outputdef.assign_output_settings import assign_output_settings
from outputdef.add_output_definitions import add_output_definitions

# The final run-sims code
from epw.run_epw_sims import simulate_all

from postproc.merge_results import merge_all_results
from other.zonelist import create_zonelist




idf_config = {
    "iddfile": "D:/EnergyPlus/Energy+.idd",
    "idf_file_path": "D:/Minimal.idf",
    "output_dir": "D:/output_IDFs"
}

CALIBRATION_STAGE = "pre_calibration"
STRATEGY = "B"
RANDOM_SEED = 42

def create_idf_for_building(building_row, building_index, scenario, calibration_stage, strategy):
    """
    Builds an IDF for a single building row. Then we save it to disk and return the file name.
    """
    IDF.setiddname(idf_config["iddfile"])
    idf = IDF(idf_config["idf_file_path"])

    # 1) Building object
    building_obj = idf.newidfobject("BUILDING")
    building_obj.Name = f"Sample_Building_{building_index}"

    orientation = building_row.get("building_orientation", 0.0)
    if not pd.isna(orientation):
        building_obj.North_Axis = orientation

    # 2) Geometry
    edge_types = []
    for side_col in ["north_side", "east_side", "south_side", "west_side"]:
        edge_types.append(building_row.get(side_col, "facade"))

    create_building_with_roof_type(
        idf,
        area=building_row["area"],
        perimeter=building_row["perimeter"],
        orientation=orientation,
        building_row=building_row,
        edge_types=edge_types,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED
    )

    # 3) Materials & Constructions
    update_construction_materials(
        idf,
        building_row,
        scenario=scenario,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED
    )
    assign_constructions_to_surfaces(idf)

    # 4) Fenestration
    add_fenestration(
        idf,
        building_row,
        scenario=scenario,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED
    )

    create_zonelist(idf, zonelist_name="ALL_ZONES")


    # 5) Lighting
    add_lights_and_parasitics(
        idf,
        building_row,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED
    )

    # 6) DHW
    add_dhw_to_idf(
        idf,
        building_row=building_row,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED,
        name_suffix=f"MyDHW_{building_index}"
    )

    # 7) HVAC (Ideal Loads)
    add_HVAC_Ideal_to_all_zones(
        idf,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED
    )

    # 8) Ventilation
    add_ventilation_to_idf(idf, building_row=building_row)

    # 9) Zone sizing + OA
    add_outdoor_air_and_zone_sizing_to_all_zones(
        idf=idf,
        building_row=building_row,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED
    )

    # 10) Ground temperatures
    add_ground_temperatures(
        idf,
        calibration_stage=calibration_stage,
        strategy=strategy,
        random_seed=RANDOM_SEED
    )

    # 11) Output definitions
    desired_vars = ["Facility Total Electric Demand Power", "Zone Air Temperature Maximum"]
    desired_meters = ["Electricity:Facility"]

    out_settings = assign_output_settings(
        desired_variables=desired_vars,
        desired_meters=desired_meters,
        override_variable_frequency="Hourly",
        override_meter_frequency="Hourly",
        include_tables=True,
        include_summary=True
    )
    add_output_definitions(idf, out_settings)

    # 12) Save IDF to disk
    os.makedirs(idf_config["output_dir"], exist_ok=True)
    idf_filename = f"building_{building_index}.idf"
    out_path = os.path.join(idf_config["output_dir"], idf_filename)
    idf.save(out_path)
    print(f"[create_idf_for_building] IDF saved at: {out_path}")

    # Return just the file name (or full path) so we can reference it later
    return idf_filename

def main():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Starting main...")

    # 1) Load building data
    df_buildings = pd.read_csv("df_buildings.csv")

    # 2) Suppose we define a list of 'desired_climate_year's. 
    #    In this example, we just store a single year per building. 
    #    If you want multiple, you can expand or replicate rows.

    # E.g., if you want to ensure every building has a 'desired_climate_year' column:
    # We'll assign 2020 for the first, 2050 for the second, etc. Or read from an array.
    desired_years = [2020, 2050]
    # for simplicity, match them row by row
    for i in range(len(df_buildings)):
        year = desired_years[i % len(desired_years)]
        df_buildings.loc[i, "desired_climate_year"] = year

    # 3) Create IDFs for each building & store the idf_name in the DF
    scenario = "scenario1"
    calibration_stage = "pre_calibration"
    strategy = "B"

    idf_names = []
    for idx, row in df_buildings.iterrows():
        print(f"--- Creating IDF for building index {idx} ---")
        idf_filename = create_idf_for_building(
            building_row=row,
            building_index=idx,
            scenario=scenario,
            calibration_stage=calibration_stage,
            strategy=strategy
        )
        # Store into df so run_epw_sims can find the file
        df_buildings.loc[idx, "idf_name"] = idf_filename

    # 4) Now we have df_buildings with columns lat, lon, desired_climate_year, idf_name
    #    We'll run them all in parallel using epw.run_epw_sims.simulate_all
    from epw.run_epw_sims import simulate_all

    # 2) Define all directories
    idf_directory = "D:/output_IDFs"  # where we saved the final .idf files for each building
    iddfile = "D:/EnergyPlus/Energy+.idd"
    base_output_dir = "D:/Sim_Results"
    num_workers = 4

    # 3) Finally, run all simulations => each year in a single folder
    simulate_all(
        df_buildings=df_buildings,
        idf_directory=idf_directory,
        iddfile=iddfile,
        base_output_dir=base_output_dir,
        num_workers=num_workers
    )

    # 3) Post-processing step
    out_csv_as_is = "D:/merged_as_is.csv"
    out_csv_daily_mean = "D:/merged_daily_mean.csv"
    
    

    # 1) Merge "as is" => no aggregation
    merge_all_results(
        base_output_dir=base_output_dir,
        output_csv=out_csv_as_is,
        convert_to_daily=False,
        convert_to_monthly=False
    )

    # 2) Convert Hourly/TimeStep to Daily => aggregator = mean
    merge_all_results(
        base_output_dir=base_output_dir,
        output_csv=out_csv_daily_mean,
        convert_to_daily=True,
        daily_aggregator="mean", # max, min, sum, pick_first_hour
        convert_to_monthly=False
    )

    print("All done post-processing.")

    
    



    logging.info("All done with main.")

if __name__ == "__main__":
    main()


--- Processing building index 0 ---
[DHW] Building key => Residential_SingleFamily_Large
     occupant_count=4 daily_liters=200.0
     setpoint=60.0C, tank=0.200 m³
     schedules => MyDHW_0_UseFraction, MyDHW_0_Setpoint
[VENTILATION] bldg_func=residential, system=C, infiltration_total=0.010 m³/s, ventilation_total=0.084 m³/s
   => Zone1: infiltration=0.0099, ventilation=0.0837
[ZoneSizing] Updated SIZING:ZONE for Zone1 with {'cooling_supply_air_temp': 14.139426798457883, 'heating_supply_air_temp': 48.100043020890666, 'cooling_supply_air_hr': 0.00877502931836912, 'heating_supply_air_hr': 0.0037232107381488227, 'cooling_design_air_flow_method': 'DesignDayWithLimit', 'heating_design_air_flow_method': 'DesignDay'}
[GROUND TEMPS] Created SITE:GROUNDTEMPERATURE:BUILDINGSURFACE with monthly values: {'January': 2.6394267984578836, 'February': 3.5375161328340003, 'March': 4.550058636738239, 'April': 7.558026845372057, 'May': 14.472942428328025, 'June': 16.176699487422912, 'July': 18.7843591354

In [1]:
import pandas as pd
import numpy as np

# Define the building data as a dictionary of lists
data_buildings = {
    "ogc_fid": [4136730, 4136731],
    "pand_id": [1.41e13, 2.51e13],
    "meestvoorkomendelabel": ["A", "B"],
    "gem_hoogte": [6.5, 8.0],
    "gem_bouwlagen": [1, 2],
    "b3_dak_type": ["multiple horizontal", "single pitched"],
    "b3_opp_dak_plat": [77.25, 50.00],
    "b3_opp_dak_schuin": [15.88000011, 25.50000000],
    "x": [230537.465, 230600.123],
    "y": [582426.737, 582500.456],
    "lon": [6.518203, 6.518500],
    "lat": [53.223112, 53.223500],
    "postcode": ["9745DZ", "9745EA"],
    "area": [93, 120],
    "perimeter": [53, 60],
    "height": [6.5, 8.0],
    "bouwjaar": [1999, 2018],
    "age_range": ["1992 - 2005", "2015 and later"],
    "average_wwr": [0.2, 0.35],
    "building_function": ["residential", "non_residential"],
    "residential_type": ["Two-and-a-half-story House", None],
    "non_residential_type": [None, "Meeting Function"],
    "north_side": ["facade", "shared"],
    "east_side": ["shared", "Facade"],
    "south_side": ["facade", "facade"],
    "west_side": ["Facade", "facade"],
    "building_orientation": [47.56956671, 135.00000000],
    "building_orientation_cardinal": ["N", "S"]
}

# Create the DataFrame
df_buildings = pd.DataFrame(data_buildings)

df_buildings.to_csv('df_buildings.csv', index=False)

# Display the DataFrame
print(df_buildings)


--- Processing building index 0 ---
[DHW] Building key => Residential_SingleFamily_Large
     occupant_count=4 daily_liters=200.0
     setpoint=60.0C, tank=0.200 m³
     schedules => MyDHW_0_UseFraction, MyDHW_0_Setpoint
[VENTILATION] bldg_func=residential, system=C, infiltration_total=0.010 m³/s, ventilation_total=0.084 m³/s
   => Zone1: infiltration=0.0099, ventilation=0.0837
Saved IDF: D:/output\building_0.idf
--- Processing building index 1 ---
[DHW] Building key => Assembly
     occupant_count=4 daily_liters=200.0
     setpoint=60.0C, tank=0.200 m³
     schedules => MyDHW_1_UseFraction, MyDHW_1_Setpoint
[VENTILATION] bldg_func=non_residential, system=D, infiltration_total=0.013 m³/s, ventilation_total=0.120 m³/s
   => Zone1_FrontPerimeter: infiltration=0.0013, ventilation=0.0120
   => Zone1_RightPerimeter: infiltration=0.0013, ventilation=0.0120
   => Zone1_RearPerimeter: infiltration=0.0013, ventilation=0.0120
   => Zone1_LeftPerimeter: infiltration=0.0013, ventilation=0.0120
   

In [ ]:
idf_config = {
    "iddfile": "D:/EnergyPlus/Energy+.idd",
    "idf_file_path": "D:/Minimal.idf",
    "output_dir": "D:/output"
}

In [7]:
from eppy import modeleditor
from eppy.modeleditor import IDF

def create_zonelist(idf, zonelist_name="ALL_ZONES"):
    # Check if a ZoneList with zonelist_name already exists
    zone_lists = idf.idfobjects["ZONELIST"]
    for zl in zone_lists:
        if zl.Name.upper() == zonelist_name.upper():
            print(f"ZoneList '{zonelist_name}' already exists.")
            return zl
    
    # Get all the Zone objects from the IDF
    zones = idf.idfobjects["ZONE"]
    if not zones:
        print("No zones found in the IDF file.")
        return None
    
    # Create a new ZONELIST object
    zonelist_obj = idf.newidfobject("ZONELIST")
    zonelist_obj.Name = zonelist_name
    
    # Loop through the zones and assign them to Zone_1_Name, Zone_2_Name, etc.
    for i, zone in enumerate(zones):
        field_name = f"Zone_{i+1}_Name"  # Use underscores
        zonelist_obj[field_name] = zone.Name
    
    print(f"ZoneList '{zonelist_name}' created with zones: {[z.Name for z in zones]}")
    return zonelist_obj

def main():
    # Paths (update these paths accordingly)
    idd_file = "D:/EnergyPlus/Energy+.idd"
    idf_file = "D:/output/building_1.idf"
    updated_idf_file = "D:/output/Building_updated.idf"
    
    # Set the IDD file for Eppy
    IDF.setiddname(idd_file)
    
    # Load the IDF file
    idf = IDF(idf_file)
    
    # Create or retrieve the ZoneList named "ALL_ZONES"
    create_zonelist(idf, zonelist_name="ALL_ZONES")
    
    # Save the updated IDF file
    idf.saveas(updated_idf_file)
    print(f"Updated IDF file saved as '{updated_idf_file}'.")

if __name__ == "__main__":
    main()


ZoneList 'ALL_ZONES' created with zones: ['Zone1_FrontPerimeter', 'Zone1_RightPerimeter', 'Zone1_RearPerimeter', 'Zone1_LeftPerimeter', 'Zone1_Core', 'Zone2_FrontPerimeter', 'Zone2_RightPerimeter', 'Zone2_RearPerimeter', 'Zone2_LeftPerimeter', 'Zone2_Core']
Updated IDF file saved as 'D:/output/Building_updated.idf'.


In [7]:
import pandas as pd

# Define the data for the new DataFrame
data_focus = {
    "focus_ogc_fid": [4136730, 4136732, 4136734, 4136734, 4136735, 4136735, 4136735],
    "edge_label": [
        "south_side",
        "south_side",
        "south_side",
        "west_side",
        "north_side",
        "south_side",
        "south_side"
    ],
    "object_id": [4136737, 4136735, 4136741, 4136741, 4136740, 4136744, 4136750],
    "object_type": ["building"] * 7,
    "object_height": [8.3, 6.3, 6.0, 6.0, 7.3, 8.3, 8.3],
    "distance_to_facade": [
        42.53512557395345,
        40.6329814404282,
        42.10188022917796,
        42.10188022917796,
        48.409795442512504,
        42.385480065525314,
        47.38134388381844
    ],
    "angle_deg": [
        287.3196238798837,
        287.1904138224335,
        182.88296993267627,
        182.88296993267627,
        142.17364298144136,
        186.4452749653224,
        227.02041617666026
    ],
    "direction": ["W", "W", "S", "S", "SE", "S", "SW"],
    "base_reflectance": [0.2] * 7,
    "base_transmittance": [0] * 7,
    "base_transmittance_summer": [0] * 7,
    "base_transmittance_winter": [0] * 7,
    "geom_28992": [
        "22",
        "22",
        "22",
        "22",
        "22",
        "22",
        "22"
    ],
    "height_diff": [1.8, 0.3, 2.8, 2.8, 1.0, 2.0, 2.0],
    "shading_score_a": [
        0.04231796605068066,
        0.007383164842083474,
        0.06650534334235053,
        0.06650534334235053,
        0.02065697635899986,
        0.04718597021687911,
        0.04221070649460908
    ],
    "shading_score_b": [
        0.04231796605068066,
        0.007383164842083474,
        0.07980641201082063,
        0.07980641201082063,
        0.02272267399489985,
        0.05662316426025493,
        0.046431777144069986
    ],
    "angle_diff": [
        17.097633484706,
        17.190391525449,
        272.718867330225,
        3.234336879605,
        52.396640737335,
        276.581187373321,
        317.156328584659
    ],
    "is_included": [True] * 7,
    "rn": [1, 1, 1, 1, 1, 1, 2]
}

# Create the new DataFrame
df_focus = pd.DataFrame(data_focus)

# Save the DataFrame to a CSV file (optional)
df_focus.to_csv('df_focus.csv', index=False)

# Display the DataFrame
print(df_focus)


   focus_ogc_fid  edge_label  object_id object_type  object_height  \
0        4136730  south_side    4136737    building            8.3   
1        4136732  south_side    4136735    building            6.3   
2        4136734  south_side    4136741    building            6.0   
3        4136734   west_side    4136741    building            6.0   
4        4136735  north_side    4136740    building            7.3   
5        4136735  south_side    4136744    building            8.3   
6        4136735  south_side    4136750    building            8.3   

   distance_to_facade   angle_deg direction  base_reflectance  \
0           42.535126  287.319624         W               0.2   
1           40.632981  287.190414         W               0.2   
2           42.101880  182.882970         S               0.2   
3           42.101880  182.882970         S               0.2   
4           48.409795  142.173643        SE               0.2   
5           42.385480  186.445275         S      

In [8]:
import pandas as pd
import ast

# Define the data for the new DataFrame
data_trees = {
    "ep_id": [1217, 5100, 5645, 362, 5124, 8513, 8799, 3573],
    "focus_ogc_fid": [4136736, 4136757, 4136760, 4136732, 4136757, 4136776, 4136777, 4136749],
    "edge_label": [
        "east_side",
        "south_side",
        "south_side",
        "south_side",
        "south_side",
        "north_side",
        "north_side",
        "north_side"
    ],
    "facade_orientation": [
        359.9999814085612,
        270.11091833609396,
        269.7769667905132,
        270.0000222969845,
        270.11091833609396,
        90.00000000244722,
        89.72430491163776,
        90.16062957332772
    ],
    "building_height": [5.8, 5.8, 8.3, 6, 5.8, 8.3, 6.2, 6.3],
    "object_id": [60081870, 60091917, 60178700, 60081870, 60178700, 60091982, 60178703, 60130596],
    "object_type": ["tree"] * 8,
    "object_height": [2.92, 2.27, 4.06, 2.92, 4.06, 2.06, 2.28, 2.2],
    "distance_to_facade": [
        19,
        40.6332924229012,
        22.8792485860304,
        37.42050746671768,
        13.32181305697539,
        4.354654462279212,
        15.465155500232099,
        7.671579417686071
    ],
    "height_diff": [
        -2.88,
        -3.53,
        -4.240000000000001,
        -3.08,
        -1.7400000000000002,
        -6.24,
        -3.9200000000000004,
        -4.1
    ],
    "shading_score_a": [0] * 8,
    "shading_score_b": [0] * 8,
    "reflectance": [0.2] * 8,
    "transmittance": [0.5] * 8,
    "transmittance_summer": [0.5] * 8,
    "transmittance_winter": [0.9] * 8,
    "angle_deg": [
        179.68403383069528,
        280.09867746829593,
        189.6612752886653,
        246.46202344815748,
        305.47888275923765,
        354.4593039991571,
        252.27569934383047,
        227.32729533393
    ],
    "direction": [
        "S",
        "W",
        "S",
        "SW",
        "NW",
        "N",
        "W",
        "SW"
    ],
    "geom_28992": [
        "11",
        "11",
        "11",
        "11",
        "11",
        "11",
        "11",
        "11"
    ],
    "vertices_local": [
        "[[0.7578125, -19, 2.92], [-0.7421875, -20, 2.92], [-0.7421875, -19, 2.92], [0.7578125, -20, 2.92], [-0.7421875, -20, 2.92]]",
        "[[-41.03125, 6.96875, 2.27], [-41.03125, 6.96875, 2.27], [-41.03125, 7.46875, 2.27], [-40.03125, 7.46875, 2.27], [-40.03125, 6.96875, 2.27]]",
        "[[-3.4773374730721116, -22.613450421369635, 4.06], [-4.477337473072112, -24.113450421369635, 4.06], [-4.477337473072112, -22.613450421369635, 4.06], [-3.4773374730721116, -24.113450421369635, 4.06], [-4.477337473072112, -24.113450421369635, 4.06]]",
        "[[-35.903927340666996, -15.71951638488099, 2.92], [-34.403927340666996, -15.71951638488099, 2.92], [-34.403927340666996, -14.71951638488099, 2.92], [-35.903927340666996, -14.71951638488099, 2.92], [-35.903927340666996, -15.71951638488099, 2.92]]",
        "[[-12.03125, 7.46875, 4.06], [-12.03125, 7.46875, 4.06], [-11.03125, 7.46875, 4.06], [-11.03125, 8.96875, 4.06], [-12.03125, 8.96875, 4.06]]",
        "[[-0.6962497416825499, 4.8502300542313606, 2.06], [-0.6962497416825499, 4.3502300542313606, 2.06], [-0.6962497416825499, 4.3502300542313606, 2.06], [-0.19624974168254994, 4.3502300542313606, 2.06], [-0.19624974168254994, 4.8502300542313606, 2.06]]",
        "[[-15.279580006521428, -5.053575460799038, 2.28], [-14.779580006521428, -5.053575460799038, 2.28], [-14.779580006521428, -4.553575460799038, 2.28], [-15.279580006521428, -4.553575460799038, 2.28], [-15.279580006521428, -5.053575460799038, 2.28]]",
        "[[-6.534155606321292, -5.812838458572514, 2.2], [-5.534155606321292, -5.812838458572514, 2.2], [-5.534155606321292, -5.312838458572514, 2.2], [-6.534155606321292, -5.312838458572514, 2.2], [-6.534155606321292, -5.812838458572514, 2.2]]"
    ],
    "active_seasons": [
        "{summer,winter}",
        "{summer,winter}",
        "{summer,winter}",
        "{summer,winter}",
        "{summer,winter}",
        "{summer,winter}",
        "{summer,winter}",
        "{summer,winter}"
    ]
}

# Create the DataFrame
df_trees = pd.DataFrame(data_trees)

# Convert 'vertices_local' from string to list of lists
df_trees['vertices_local'] = df_trees['vertices_local'].apply(ast.literal_eval)

# Convert 'active_seasons' from string to set
df_trees['active_seasons'] = df_trees['active_seasons'].apply(lambda x: set(x.strip("{}").split(",")))

# Save the DataFrame to a CSV file (optional)
df_trees.to_csv('df_trees.csv', index=False, encoding='utf-8-sig')

# Display the DataFrame
print(df_trees)


   ep_id  focus_ogc_fid  edge_label  facade_orientation  building_height  \
0   1217        4136736   east_side          359.999981              5.8   
1   5100        4136757  south_side          270.110918              5.8   
2   5645        4136760  south_side          269.776967              8.3   
3    362        4136732  south_side          270.000022              6.0   
4   5124        4136757  south_side          270.110918              5.8   
5   8513        4136776  north_side           90.000000              8.3   
6   8799        4136777  north_side           89.724305              6.2   
7   3573        4136749  north_side           90.160630              6.3   

   object_id object_type  object_height  distance_to_facade  height_diff  ...  \
0   60081870        tree           2.92           19.000000        -2.88  ...   
1   60091917        tree           2.27           40.633292        -3.53  ...   
2   60178700        tree           4.06           22.879249        -4.24